In [1]:
import cv2
from PIL import Image

import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
train_cats = '/home/admin1/opencv/Convolutional-Neural-Networks/Convolutional_Neural_Networks/dataset/training_set/cats'
train_dogs = '/home/admin1/opencv/Convolutional-Neural-Networks/Convolutional_Neural_Networks/dataset/training_set/dogs'

test_cats = '/home/admin1/opencv/Convolutional-Neural-Networks/Convolutional_Neural_Networks/dataset/test_set/cats'
test_dogs = '/home/admin1/opencv/Convolutional-Neural-Networks/Convolutional_Neural_Networks/dataset/test_set/dogs'

In [3]:
print('Number of images of Cats in training dataset are-', len(os.listdir(train_cats)))
print('Number of images of Dogs in training dataset are-', len(os.listdir(train_dogs)))
print('Number of images of Cats in test dataset are-', len(os.listdir(test_cats)))
print('Number of images of Cats in ttest dataset are-', len(os.listdir(test_dogs)))

Number of images of Cats in training dataset are- 4001
Number of images of Dogs in training dataset are- 4001
Number of images of Cats in test dataset are- 1001
Number of images of Cats in ttest dataset are- 1001


# Initialising the CNN

In [4]:
model = tf.keras.Sequential()

# Convolution Layer and MaxPooling Layer

In [5]:
model.add(tf.keras.layers.Convolution2D(filters = 32, kernel_size = (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
model.add(tf.keras.layers.Convolution2D(filters = 32, kernel_size = (3, 3), activation = 'relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Convolution2D(filters = 40, kernel_size = (3, 3), activation = 'relu'))
model.add(tf.keras.layers.Convolution2D(filters = 32, kernel_size = (3, 3), activation = 'relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))
model.add(tf.keras.layers.Dropout(0.5))

# Flattening

In [6]:
model.add(tf.keras.layers.Flatten())

# Fully Connected Layer

In [7]:
model.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))

# Output Layer

In [8]:
model.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Compiling the model

In [9]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 60, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 30, 30, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 40)        11560     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 32)        11552     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0

# Fitting the CNN to the Images

### Image Augmentaion

# Training Dataset

In [11]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                                               shear_range = 0.2,
                                                               zoom_range = 0.2,
                                                               horizontal_flip = True)

In [12]:
training_set = train_datagen.flow_from_directory(directory ='/home/admin1/opencv/Convolutional-Neural-Networks/Convolutional_Neural_Networks/dataset/training_set',
                                                target_size = (64,64),
                                                batch_size = 250,
                                                class_mode = 'binary')

Found 8000 images belonging to 2 classes.


In [13]:
len(training_set)

32

# Test Dataset

In [14]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

In [15]:
test_set = test_datagen.flow_from_directory(directory = '/home/admin1/opencv/Convolutional-Neural-Networks/Convolutional_Neural_Networks/dataset/test_set',
                                           target_size = (64, 64),
                                           batch_size = 250,
                                           class_mode = 'binary')

Found 2000 images belonging to 2 classes.


In [16]:
len(test_set)

8